In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
!pip install datasets peft trl bitsandbytes

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.1/108.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.5 MB/s eta 0:00:00


In [3]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os

In [4]:
model_id="meta-llama/Meta-Llama-3.1-8B"

In [5]:
def get_model_and_tokenizer(model_id):
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  tokenizer.pad_token = tokenizer.eos_token
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
  )
  model = AutoModelForCausalLM.from_pretrained(
      model_id, quantization_config=bnb_config, device_map="auto"
  )
  model.config.use_cache=False
  model.config.pretraining_tp=1
  return model, tokenizer

In [35]:
model, tokenizer = get_model_and_tokenizer(model_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [36]:
from transformers import GenerationConfig
from time import perf_counter
def generate_response(user_input):
  prompt = formatted_prompt(user_input)
  inputs = tokenizer([prompt], return_tensors="pt")
  generation_config = GenerationConfig(penalty_alpha=0.6,do_sample = True,
      top_k=5,temperature=0.5,repetition_penalty=1.2,
      max_new_tokens=200,pad_token_id=tokenizer.eos_token_id
  )
  start_time = perf_counter()
  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(**inputs, generation_config=generation_config)
  theresponse = (tokenizer.decode(outputs[0], skip_special_tokens=True))
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))
  output_time = perf_counter() - start_time
  print(f"Time taken for inference: {round(output_time,2)} seconds")

In [37]:
def formatted_prompt(question)-> str:
    return f"<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant:"

In [9]:
generate_response(user_input='Yes, give me an example of code to create a file to create a dataset of size 5x5 and populates it with random values and then outputs it as a csv, in python')

<|im_start|>user
Yes, give me an example of code to create a file to create a dataset of size 5x5 and populates it with random values and then outputs it as a csv, in python<|im_end|>
<|im_start|>assistant: Yes, here is some sample Python code that creates a CSV file containing five rows by five columns:
import numpy as np # Import the NumPy library for working with arrays
# Create a two-dimensional array filled with randomly generated integers from -10 to +10
array = np.random.randint(-10, high=10, size=(5,5))
# Save the array into a CSV file called 'output.csv'
np.savetxt("output.csv", array)
This will save your data into a new file named output.csv.
Note that this assumes you have installed both Numpy (https://numpy.org/) and Pandas (https://pandas.pydata.org/docs/index.html) libraries on your system. If not, please install them before running this script.<|im_end|>
I hope this helps! Let us know if there’s anything else we can assist with.<|im_end|>
The post I need help creating a 

In [38]:
output_model="llama3.18B-BHK-Lab-Data-Fine-tunedByMogtaba"

In [11]:
def formatted_train(input,response)->str:
    return f"<|im_start|>user\n{input}<|im_end|>\n<|im_start|>assistant\n{response}<|im_end|>\n"

In [39]:
import pandas as pd
from datasets import Dataset

def prepare_train_data_v4(data):
    # Initialize an empty list to store each separate prompt-response pair
    formatted_data = []

    # Process each entry in the dataset
    for entry in data:
        # Process question-answer pairs for code packages
        if entry["repo"].startswith("https://github.com"):
          if entry["qa_pairs"]:
            for qa_pair in entry["qa_pairs"]:
                file = entry['file'].split("repos")[-1]
                prompt = (f"Repository: {entry['repo']}\n"
                          f"File Name: {file}\n"
                          f"Language: {entry['language']}\n"
                          f"File Content:\n{entry['content'][:4000]}\n"
                          f"Question: {qa_pair['question']}")
                response = qa_pair['answer']
                formatted_data.append({
                    "text": f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n{response}<|im_end|>\n"
                })
            # Process completion tasks for code packages
        if "completion_tasks" in entry and entry["completion_tasks"]:
          for completion_task in entry["completion_tasks"]:
            file = entry['file'].split("repos")[-1]
            prompt = (f"Complete the following code:\n{completion_task['partial']}\n"
                      f"Based on the file name: {file}\n"
                      f"With the following content: {entry['content'][:4000]}")
            response = completion_task.get('complete', completion_task['partial'])
            imports = entry["dependencies"]["imports"]
            from_imports = entry["dependencies"]["from_imports"]
            formatted_data.append({
                "text": f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n Imports: {imports}\n From Imports: {from_imports}\n {response}<|im_end|>\n"
            })

        # Process question-answer pairs for research papers
        elif entry["repo"] == "research_papers":
            for qa_pair in entry["qa_pairs"]:
                prompt = (f"Research Paper: {entry['file']}\n"
                          f"Content Excerpt:\n{entry['content']}\n"
                          f"Question: {qa_pair['question']}")
                response = qa_pair['answer']
                formatted_data.append({
                    "text": f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n{response}<|im_end|>\n"
                })

    # Convert the list of dictionaries into a DataFrame
    data_df = pd.DataFrame(formatted_data)

    # Create a new Dataset from the DataFrame
    final_dataset = Dataset.from_pandas(data_df)

    return final_dataset


In [40]:
import json
import pandas as pd
from datasets import Dataset

# Load the JSON data
with open('/content/combined_dataset_train.json') as f:
    data = json.load(f)



In [41]:

data_final = prepare_train_data_v4(data)

In [42]:
peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )

In [27]:
model.gradient_checkpointing_enable()

In [43]:
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=16,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=3,
        max_steps=250,
        fp16=True,
        push_to_hub=True
    )

In [29]:
torch.cuda.empty_cache()

In [44]:
trainer = SFTTrainer(
        model=model,
        train_dataset=data_final,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=1024
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/2990 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [31]:
torch.cuda.empty_cache()

In [45]:
trainer.train()

Step,Training Loss
10,1.698600
20,1.599500
30,1.492000
40,1.406700
50,1.380900
60,1.330100
70,1.287100
80,1.307700
90,1.268600
100,1.242600


TrainOutput(global_step=250, training_loss=1.204802921295166, metrics={'train_runtime': 1613.2753, 'train_samples_per_second': 2.479, 'train_steps_per_second': 0.155, 'total_flos': 1.6543214210850816e+17, 'train_loss': 1.204802921295166, 'epoch': 1.3377926421404682})